In [37]:
#import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD, Adam
from keras.callbacks import EarlyStopping
from keras import backend
from keras.layers.wrappers import Bidirectional
from keras.layers.recurrent import LSTM

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf

In [17]:
n_in = 39
n_time = 20
n_hidden = 128
n_out = 20

epochs = 300
batch_size = 250

In [8]:
log_mini= pd.read_csv("log_mini_finished.csv")
songs= log_mini.drop(["session_id",'y1', 'y2'], axis=1)

X_train = np.array(songs)
#Y_train = np.array(log_mini['y2'])

X_train_c = []
for i in range(int(len(log_mini)/20)):
    X_train_c.append(X_train[20*i:20*(i+1), :])

X_train = np.array(X_train_c)
#X_test = X_test.reshape(X_test.shape[0], 28, 28)

In [18]:
Y_train = np.array(log_mini['y2'])
Y_train_c = []
for i in range(int(len(log_mini)/20)):
    Y_train_c.append(Y_train[20*i:20*(i+1)])
Y_train = np.array(Y_train_c)

In [19]:
print(X_train.shape, Y_train.shape)

(10000, 20, 39) (10000, 20)


In [12]:
def weight_variable(shape, name=None):
    return np.random.normal(scale=.01, size=shape)

In [38]:
def selective_hinge(y_true, y_pred):
    return K.mean(K.maximum(1. - y_true * y_pred, 0.) * K.pow(y_true,2), axis=-1)

In [39]:
def selective_binary_accuracy(y_true, y_pred):
    return K.mean(K.equal(y_true, K.sign(y_pred)), axis=-1)

In [53]:
from keras.models import load_model

model = Sequential()
model.add(LSTM(n_hidden, input_shape=(n_time, n_in)))
model.add(Dense(n_out, kernel_initializer=weight_variable))
model.add(Activation('linear'))

#
#model = load_model("model.h5", custom_objects={'selective_hinge': selective_hinge})

model.compile(loss='hinge',
              optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999),
              metrics=[selective_binary_accuracy])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_19 (LSTM)               (None, 128)               86016     
_________________________________________________________________
dense_18 (Dense)             (None, 20)                2580      
_________________________________________________________________
activation_18 (Activation)   (None, 20)                0         
Total params: 88,596
Trainable params: 88,596
Non-trainable params: 0
_________________________________________________________________


In [54]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1)

hist = model.fit(X_train, Y_train,
                 batch_size=batch_size,
                 epochs=epochs,
                 validation_data=(X_train, Y_train),
                 callbacks=[early_stopping])

Train on 10000 samples, validate on 10000 samples
Epoch 1/300
10000/10000 [==============================] - 7s 742us/step - loss: 0.9814 - selective_binary_accuracy: 0.2249 - val_loss: 0.9693 - val_selective_binary_accuracy: 0.2321
Epoch 2/300
10000/10000 [==============================] - 6s 594us/step - loss: 0.9632 - selective_binary_accuracy: 0.2355 - val_loss: 0.9543 - val_selective_binary_accuracy: 0.2371
Epoch 3/300
10000/10000 [==============================] - 6s 635us/step - loss: 0.9546 - selective_binary_accuracy: 0.2373 - val_loss: 0.9513 - val_selective_binary_accuracy: 0.2384
Epoch 4/300
10000/10000 [==============================] - 6s 595us/step - loss: 0.9507 - selective_binary_accuracy: 0.2392 - val_loss: 0.9500 - val_selective_binary_accuracy: 0.2398
Epoch 5/300
10000/10000 [==============================] - 6s 600us/step - loss: 0.9494 - selective_binary_accuracy: 0.2400 - val_loss: 0.9483 - val_selective_binary_accuracy: 0.2403
Epoch 6/300
10000/10000 [==========

KeyboardInterrupt: 